# Examples


In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator

from src.bearing_calculations import solve_eccentricity, solve_K_and_C_Friswell, solve_K_and_C_AlBender
from src.pressure_distribution import pressure_distribution
from src.compressible_flow import compressible_flow
from src.modal_analysis import Shaft, Bearing, RotorSystem

## Task 1

Calculate K and C matrixes for fluid bearing (journal/tilting pad)

### Friswell example 5.5


In [ ]:
# Given parameters
D = 100  # mm, diameter
L = 30  # mm, length
f = 525  # N, static load
c = 0.1  # mm, radial clearance
omega = 1500  # rpm, rotational speed
eta = 0.1  # Pa.s, viscosity

# Necessary unit conversions
D = D / 1000  # Convert mm to m
L = L / 1000  # Convert mm to m
c = c / 1000  # Convert mm to m
omega = omega * (2 * np.pi) / 60  # Convert rpm to rad/s

epsilon = solve_eccentricity(D, omega, eta, L, f, c)
print("Eccentricity (epsilon):", epsilon)

K, C = solve_K_and_C_Friswell(omega, f, c, epsilon)

print()
print("Stiffness matrix K (N/m):")
print(K)

print()
print("Damping matrix C (Ns/m):")
print(C)

In [ ]:
# Book answers are in MN/m and kNs/m

print()
print("Stiffness matrix K (MN/m):")
print(K / 1e6)

print()
print("Damping matrix C (kNs/m):")
print(C / 1e3)

### Al-Bender figures 9.5 and 9.6


In [ ]:
# Helpers


def plot_dynamic_coeff(sigma_vals, vals, ylabel, ax):
    ax.semilogx(sigma_vals, vals)

    ax.set_ylabel(f"${ylabel}$")
    ax.set_xlabel("σ")

    # Major ticks
    ax.xaxis.set_major_locator(LogLocator(base=10))
    ax.grid(which="major", linestyle="-", color="gray", alpha=0.5)

    # Minor ticks
    ax.xaxis.set_minor_locator(LogLocator(base=10, subs="auto"))
    ax.grid(which="minor", linestyle="--", color="lightgray", alpha=0.5)

    return ax

In [ ]:
fig1, axs = plt.subplots(2, 2, sharex=True, figsize=(12, 8))
axs = axs.ravel()

lambda_vals = [0.1, 1, 5, 10, 100, 1000]
sigma_vals = np.logspace(-1, 3, 10000)
LD_ratio = 1

for lambda_ in lambda_vals:
    K, C = solve_K_and_C_AlBender(lambda_, sigma_vals, LD_ratio)

    K_xx = K[0, 0]
    K_xy = K[0, 1]
    C_xx = C[0, 0]
    C_xy = C[0, 1]

    plot_dynamic_coeff(sigma_vals, K_xx, "K_{xx}", axs[0])
    axs[0].legend(title="Λ", loc="lower right", labels=lambda_vals)
    plot_dynamic_coeff(sigma_vals, K_xy, "K_{xy}", axs[1])
    plot_dynamic_coeff(sigma_vals, C_xx, "C_{xx}", axs[2])
    plot_dynamic_coeff(sigma_vals, C_xy, "C_{xy}", axs[3])

plt.suptitle("Al-Bender Figures 9.5 (top) and 9.6 (bottom)")
plt.tight_layout()
plt.show()

## Task 3

### Almqvist 5.2


In [ ]:
mu = 0.04
omega = 2 * np.pi
r_in = 0.05
r_out = 0.10
n_sector = 6
Delta_theta = np.pi / 4
hL = 50e-6
hT = 10e-6

pressure_distribution(
    mu,
    omega,
    r_in,
    r_out,
    n_sector,
    Delta_theta,
    hL,
    hT,
    # savefig=True,
    # out_figure="pressure_contours_new.png",
)

## Task 4


In [ ]:
mu = 0.04
omega = 2 * np.pi
r_in = 0.05
r_out = 0.10
n_sector = 6
Delta_theta = np.pi / 4
hL = 50e-6
hT = 10e-6
boundary = True
rho_a = 1.2
p_a = 101325
beta = 1e7
max_iter = 80
tol = 1e-6
relax = 0.4

compressible_flow(
    mu=mu,
    omega=omega,
    r_in=r_in,
    r_out=r_out,
    n_sector=n_sector,
    Delta_theta=Delta_theta,
    hL=hL,
    hT=hT,
    boundary=boundary,
    rho_a=rho_a,
    p_a=p_a,
    beta=beta,
    max_iter=max_iter,
    tol=tol,
    relax=relax,
    # savefig=True,
    # out_figure="compressible_flow_NEW.png",
)

## Task 7

### Friswell example 6.8.1


In [ ]:
# Bearings at node 0 and 4
bearing_1 = Bearing(node=0, kxx=0.2e6, kyy=0.4e6, cxx=0.0, cyy=0.0)
bearing_2 = Bearing(node=4, kxx=0.2e6, kyy=0.4e6, cxx=0.0, cyy=0.0)
bearings = [bearing_1, bearing_2]

rotor = RotorSystem(bearings=bearings)
speeds = np.linspace(0, 3000, 1000)
rotor.campbell(speeds)
rotor.print_eigs()

## Ross integration


In [ ]:
import ross as rs
import plotly.graph_objects as go

# Make sure the default renderer is set to 'notebook' for inline plots in Jupyter
import plotly.io as pio

pio.renderers.default = "notebook"

Q_ = rs.Q_

steel = rs.Material("steel", E=211e9, G_s=81.2e9, rho=7810)

In [ ]:
L = 0.25
N = 6
idl = 0
odl = 0.05

shaft = [rs.ShaftElement(L=L, idl=idl, odl=odl, material=steel) for i in range(N)]
bearings = [
    rs.BearingElement(n=0, kxx=1e7, cxx=0),
    rs.BearingElement(n=4, kxx=1e7, cxx=0),
]
disks = [
    rs.DiskElement.from_geometry(
        n=N,
        material=steel,
        width=0.07,
        i_d=odl,
        o_d=0.35,
    ),
]

rotor = rs.Rotor(shaft_elements=shaft, disk_elements=disks, bearing_elements=bearings)
rotor.plot_rotor()

In [ ]:
campbell = rotor.run_campbell(speed_range=Q_(list(range(0, 4500, 50)), "RPM"))

In [ ]:
campbell.plot(frequency_units="RPM", harmonics=[1, 2, 3])

In [ ]:
modal = rotor.run_modal(speed=Q_(4000, "RPM"))

In [ ]:
for mode in range(6):
    display(modal.plot_mode_3d(mode, frequency_units="Hz"))

In [ ]:
for mode in range(6):
    display(modal.plot_orbit(mode, nodes=[2, 4]))